In [5]:
!pip install transformers
!pip install note_seq
!pip install pygame
!pip install torch==2.0.1
!pip install miditoolkit # MidiFile() 로 midi file 읽어오기 위한 라이브러리

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 7.5 MB/s eta 0:00:00
  Using cached pyparsing-3.1.1-py3-none-any.whl.metadata (5.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 32.5 MB/s eta 0:00:0000:0100:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 5.6 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 8.3 MB/s eta 0:00:00
Using cached pyparsing-3.1.1-py3-none-any.whl (103 kB)


In [31]:
import os
import pygame

from copy import deepcopy
from math import ceil
from pathlib import Path
from miditoolkit import MidiFile
import pandas as pd

In [3]:
MAX_NB_BAR = 4
MIN_NB_NOTES = 20

In [35]:
midi_paths = list(Path('./ym-midis/midis').glob('*.mid'))

In [37]:
orig_names = [path.stem for path in midi_paths]

midi_data = pd.DataFrame(columns=['orig_name'], data=orig_names)
midi_data

,orig_name
0,Out Run 3D (FM) - 05 - Last Wave
1,18 Darkmare
2,Solomon no Kagi (FM) - 01 - Main Theme
3,09 Glory
4,Out Run (FM) - 03 - Splash Wave
...,...
664,Double Dragon (FM) - 07 - Last Boss
665,18 Ending 3
666,11 Your Rank
667,"12 Game Over (Namco Logo, The Tower of Druaga)"


In [38]:
# replacte - with _
replaced_paths = [str(midi_path).replace('-', '_') for midi_path in orig_paths]
# replace white space with underscore
replaced_paths = [str(midi_path).replace(' ', '_') for midi_path in replaced_paths]
# replace continuos _ with single _
replaced_paths = [str(midi_path).replace('__', '_') for midi_path in replaced_paths]
replaced_paths = [str(midi_path).replace('__', '_') for midi_path in replaced_paths]
replaced_paths = [str(midi_path).replace('__', '_') for midi_path in replaced_paths]

midi_data['replaced_name'] = replaced_paths
midi_data

,orig_name,replaced_name
0,Out Run 3D (FM) - 05 - Last Wave,Out_Run_3D_(FM)_05_Last_Wave
1,18 Darkmare,18_Darkmare
2,Solomon no Kagi (FM) - 01 - Main Theme,Solomon_no_Kagi_(FM)_01_Main_Theme
3,09 Glory,09_Glory
4,Out Run (FM) - 03 - Splash Wave,Out_Run_(FM)_03_Splash_Wave
...,...,...
664,Double Dragon (FM) - 07 - Last Boss,Double_Dragon_(FM)_07_Last_Boss
665,18 Ending 3,18_Ending_3
666,11 Your Rank,11_Your_Rank
667,"12 Game Over (Namco Logo, The Tower of Druaga)","12_Game_Over_(Namco_Logo,_The_Tower_of_Druaga)"


In [42]:
# sort by replaced_name
midi_data = midi_data.sort_values(by='replaced_name')

# reset index
midi_data = midi_data.reset_index(drop=True)
midi_data

,orig_name,replaced_name
0,01 A Ball of Light,01_A_Ball_of_Light
1,01 Compile,01_Compile
2,01 - Game de check! Koutsuu Anzen (FM) - Instr...,01_Game_de_check!_Koutsuu_Anzen_(FM)_Instructions
3,01 Hyper Defending Force (Title),01_Hyper_Defending_Force_(Title)
4,01 Is it Domingo Today,01_Is_it_Domingo_Today
...,...,...
664,Zillion II (FM) - 01 - Title Screen,Zillion_II_(FM)_01_Title_Screen
665,Zillion II (FM) - 02 - Pure Stone,Zillion_II_(FM)_02_Pure_Stone
666,Zillion II (FM) - 03 - Inside Base,Zillion_II_(FM)_03_Inside_Base
667,Zillion II (FM) - 04 - Boss,Zillion_II_(FM)_04_Boss


In [16]:
from mido import MidiFile

for i, midi_path in enumerate(midi_paths):  
    # avoid any Error
    try:
        midi = MidiFile(midi_path)
    except Exception as e:
        print("Skipping", midi_path, "because it's too short")
        continue
    
    ticks_per_cut = MAX_NB_BAR * midi.ticks_per_beat * 4
    nb_cut = ceil(midi.max_tick / ticks_per_cut)
    if nb_cut < 2:
        # if there is no midi_path.setm folder, create it
        if not os.path.exists(f'./ym-test/chunks/{midi_path.stem}'):
            os.makedirs(f'./ym-test/chunks/{midi_path.stem}')
        midi.dump(f'./ym-test/chunks/{midi_path.stem}/0.mid')
        print("Skipping", midi_path, "because it's too short")
        continue

    midi_cuts = [deepcopy(midi) for _ in range(nb_cut)]

    for j, track in enumerate(midi.instruments):
        track.notes = sorted(track.notes, key=lambda x: x.start)
        for midi_short in midi_cuts:
            midi_short.instruments[j].notes = []
        for note in track.notes:
            cut_idx = note.start // ticks_per_cut
            note_copy = deepcopy(note)
            note_copy.start -= cut_idx * ticks_per_cut
            note_copy.end -= cut_idx * ticks_per_cut
            midi_cuts[cut_idx].instruments[j].notes.append(note_copy)

    # saving midis
    for j, midi_short in enumerate(midi_cuts):
        if sum(len(track.notes) for track in midi_short.instruments) < MIN_NB_NOTES:
            print("Skipping", midi_path, "because it's too short")
            continue
        if not os.path.exists(f'./ym-test/chunks/{midi_path.stem}'):
            os.makedirs(f'./ym-test/chunks/{midi_path.stem}')
        midi_short.dump(f'./ym-test/chunks/{midi_path.stem}/{j}.mid')
            

Skipping ym-test/01 A Ball of Light.mid because it's too short
Skipping ym-test/01 A Ball of Light.mid because it's too short
Skipping ym-test/01 Hyper Defending Force (Title).mid because it's too short
Skipping ym-test/01 Hyper Defending Force (Title).mid because it's too short
Skipping ym-test/01 Hyper Defending Force (Title).mid because it's too short
Skipping ym-test/01 Hyper Defending Force (Title).mid because it's too short
Skipping ym-test/01 Hyper Defending Force (Title).mid because it's too short
Skipping ym-test/01 Hyper Defending Force (Title).mid because it's too short
Skipping ym-test/01 Hyper Defending Force (Title).mid because it's too short
Skipping ym-test/01 Hyper Defending Force (Title).mid because it's too short
Skipping ym-test/01 Hyper Defending Force (Title).mid because it's too short
Skipping ym-test/01 Hyper Defending Force (Title).mid because it's too short
Skipping ym-test/01 Hyper Defending Force (Title).mid because it's too short
Skipping ym-test/01 Hyper D

In [13]:
orig = MidiFile('./ym-test/01 - Game de check! Koutsuu Anzen (FM) - Instructions.mid')
orig.instruments

[Instrument(program=27, is_drum=False, name=) - 68 notes,
 Instrument(program=32, is_drum=False, name=) - 114 notes,
 Instrument(program=11, is_drum=False, name=) - 128 notes,
 Instrument(program=81, is_drum=False, name=) - 68 notes,
 Instrument(program=11, is_drum=False, name=) - 128 notes]

In [14]:
orig.dump('./ym-test/test.mid')

In [15]:
test = MidiFile('./ym-test/test.mid')
test.instruments

[Instrument(program=27, is_drum=False, name=) - 68 notes,
 Instrument(program=32, is_drum=False, name=) - 114 notes,
 Instrument(program=11, is_drum=False, name=) - 128 notes,
 Instrument(program=81, is_drum=False, name=) - 68 notes,
 Instrument(program=11, is_drum=False, name=) - 128 notes]

In [1]:
midi_paths

NameError: name 'midi_paths' is not defined

In [29]:
midi_paths = list(Path('./ym-test').glob('*.mid'))
midi_paths.append(Path('./ym-test/gilbert hello #3 - Rickard.mid'))
midi_paths.append(Path('./ym-test/double  space.mid'))

midi_paths = list(Path('./YM2413-MDB-v1.0.0/midi/adjust_tempo').glob('*.mid'))
midi_paths

[PosixPath('YM2413-MDB-v1.0.0/midi/adjust_tempo/Out Run 3D (FM) - 05 - Last Wave.mid'),
 PosixPath('YM2413-MDB-v1.0.0/midi/adjust_tempo/18 Darkmare.mid'),
 PosixPath('YM2413-MDB-v1.0.0/midi/adjust_tempo/Solomon no Kagi (FM) - 01 - Main Theme.mid'),
 PosixPath('YM2413-MDB-v1.0.0/midi/adjust_tempo/09 Glory.mid'),
 PosixPath('YM2413-MDB-v1.0.0/midi/adjust_tempo/Out Run (FM) - 03 - Splash Wave.mid'),
 PosixPath('YM2413-MDB-v1.0.0/midi/adjust_tempo/Megumi Rescue (FM) - 08 - Rounds 21-30.mid'),
 PosixPath('YM2413-MDB-v1.0.0/midi/adjust_tempo/Phantasy Star (FM) - 05 - Palma.mid'),
 PosixPath('YM2413-MDB-v1.0.0/midi/adjust_tempo/Shanghai (FM) - 06 - Well Done.mid'),
 PosixPath('YM2413-MDB-v1.0.0/midi/adjust_tempo/Golvellius - Valley of Doom (FM) - 16 - Overworld 3.mid'),
 PosixPath('YM2413-MDB-v1.0.0/midi/adjust_tempo/Wonder Boy III (FM) - 01 - The Last Dungeon.mid'),
 PosixPath('YM2413-MDB-v1.0.0/midi/adjust_tempo/11 Calm Feeling.mid'),
 PosixPath('YM2413-MDB-v1.0.0/midi/adjust_tempo/Out Run 

In [30]:
# replacte - with _
midi_paths = [str(midi_path.stem).replace('-', '_') for midi_path in midi_paths]
# replace white space with underscore
midi_paths = [str(midi_path).replace(' ', '_') for midi_path in midi_paths]
# replace continuos _ with single _
midi_paths = [str(midi_path).replace('__', '_') for midi_path in midi_paths]
midi_paths = [str(midi_path).replace('__', '_') for midi_path in midi_paths]
midi_paths = [str(midi_path).replace('__', '_') for midi_path in midi_paths]
midi_paths

['Out_Run_3D_(FM)_05_Last_Wave',
 '18_Darkmare',
 'Solomon_no_Kagi_(FM)_01_Main_Theme',
 '09_Glory',
 'Out_Run_(FM)_03_Splash_Wave',
 'Megumi_Rescue_(FM)_08_Rounds_21_30',
 'Phantasy_Star_(FM)_05_Palma',
 'Shanghai_(FM)_06_Well_Done',
 'Golvellius_Valley_of_Doom_(FM)_16_Overworld_3',
 'Wonder_Boy_III_(FM)_01_The_Last_Dungeon',
 '11_Calm_Feeling',
 'Out_Run_3D_(FM)_02_Midnight_Highway',
 '09_Mission_3',
 '03_Pit_Approach',
 'Global_Defense_(FM)_01_Demo',
 'Double_Dragon_(FM)_04_Mission_3',
 'Megumi_Rescue_(FM)_04_Perfect',
 '01_Title',
 '21_Mission_9',
 '08_Game_de_check!_Koutsuu_Anzen_(FM)_Game_Over',
 '20_Game_Over',
 '22_Ranking',
 '07_Ruins',
 'Power_Strike,_Aleste_(FM)_10_Dogodaga_Dogodege',
 '09_Zeodalley',
 '06_Area_2_(Lost_World_The_Island)',
 '14_Clear_#2',
 '08_Big_Don',
 '04_Turn_on_The_Run',
 '11_Complete',
 'Captain_Silver_(FM)_06_Scene_Complete',
 '10_The_Devastation_(Area_4)',
 'Galactic_Protector_(FM)_07_Ending',
 'Galactic_Protector_(FM)_06_Perfect!',
 '11_The_Solitude_